In [13]:
import pandas as pd

# Cargar el CSV desde la carpeta data
df = pd.read_csv("data/covid_espana.csv")

# Ver las primeras filas
df.head()

,provincia_iso,sexo,grupo_edad,fecha,num_casos,num_hosp,num_uci,num_def
0,A,H,0-9,2020-01-01,0,0,0,0
1,A,H,10-19,2020-01-01,0,0,0,0
2,A,H,20-29,2020-01-01,0,0,0,0
3,A,H,30-39,2020-01-01,0,0,0,0
4,A,H,40-49,2020-01-01,0,0,0,0


In [14]:
df["provincia_iso"].unique()

array(['A', 'AB', 'AL', 'AV', 'B', 'BA', 'BI', 'BU', 'C', 'CA', 'CC',
       'CE', 'CO', 'CR', 'CS', 'CU', 'GC', 'GI', 'GR', 'GU', 'H', 'HU',
       'J', 'L', 'LE', 'LO', 'LU', 'M', 'MA', 'ML', 'MU', nan, 'NC', 'O',
       'OR', 'P', 'PM', 'PO', 'S', 'SA', 'SE', 'SG', 'SO', 'SS', 'T',
       'TE', 'TF', 'TO', 'V', 'VA', 'VI', 'Z', 'ZA'], dtype=object)

In [18]:
mapa_comunidades.update({

    'A': 'Comunidad Valenciana',     # Alicante
    'AB': 'Castilla-La Mancha',      # Albacete
    'AL': 'Andalucía',               # Almería
    'AV': 'Castilla y León',         # Ávila
    'B': 'Cataluña',                 # Barcelona
    'BA': 'Extremadura',             # Badajoz
    'BI': 'País Vasco',              # Bizkaia
    'BU': 'Castilla y León',         # Burgos
    'C': 'Galicia',                  # A Coruña
    'CA': 'Andalucía',               # Cádiz
    'CC': 'Extremadura',             # Cáceres
    'CE': 'Ceuta y Melilla',         # Ceuta
    'CO': 'Andalucía',               # Córdoba
    'CR': 'Castilla-La Mancha',      # Ciudad Real
    'CS': 'Comunidad Valenciana',    # Castellón
    'CU': 'Castilla-La Mancha',      # Cuenca
    'GC': 'Canarias',                # Gran Canaria (Las Palmas)
    'GI': 'Cataluña',                # Girona
    'GR': 'Andalucía',               # Granada
    'GU': 'Castilla-La Mancha',      # Guadalajara
    'H': 'Andalucía',                # Huelva
    'HU': 'Aragón',                  # Huesca
    'J': 'Andalucía',                # Jaén
    'L': 'Cataluña',                 # Lleida
    'LE': 'Castilla y León',         # León
    'LO': 'La Rioja',                # La Rioja
    'LU': 'Galicia',                 # Lugo
    'M': 'Madrid',                   # Madrid
    'MA': 'Andalucía',               # Málaga
    'ML': 'Ceuta y Melilla',         # Melilla
    'MU': 'Murcia',                  # Murcia
    'NC': 'Navarra',                 # Navarra (equivalente a NA)
    'O': 'Asturias',                 # Asturias
    'OR': 'Galicia',                 # Ourense
    'P': 'Castilla y León',          # Palencia
    'PM': 'Baleares',                # Islas Baleares (Palma de Mallorca)
    'PO': 'Galicia',                 # Pontevedra
    'S': 'Cantabria',                # Cantabria
    'SA': 'Castilla y León',         # Salamanca
    'SE': 'Andalucía',               # Sevilla
    'SG': 'Castilla y León',         # Segovia
    'SO': 'Castilla y León',         # Soria
    'SS': 'País Vasco',              # Gipuzkoa
    'T': 'Cataluña',                 # Tarragona
    'TE': 'Aragón',                  # Teruel
    'TF': 'Canarias',                # Tenerife
    'TO': 'Castilla-La Mancha',      # Toledo
    'V': 'Comunidad Valenciana',     # Valencia
    'VA': 'Castilla y León',         # Valladolid
    'VI': 'País Vasco',              # Álava
    'Z': 'Aragón',                   # Zaragoza
    'ZA': 'Castilla y León'          # Zamora
})



In [19]:
df = df.dropna(subset=["provincia_iso"])


In [20]:
df["comunidad"] = df["provincia_iso"].map(mapa_comunidades)


In [7]:
df[["provincia_iso", "comunidad"]].drop_duplicates().sort_values("comunidad")


,provincia_iso,comunidad
540,GR,Andalucía
60,AL,Andalucía
1200,SE,Andalucía
840,MA,Andalucía
660,J,Andalucía
270,CA,Andalucía
360,CO,Andalucía
600,H,Andalucía
1530,Z,Aragón
630,HU,Aragón
